In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Download Dataset**

In [ ]:
# get dataset from huggingface hub
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/datasets/LeoFeng/MLHW_6
!unzip ./MLHW_6/faces.zip -d .

# **StyleGan2**

In [ ]:
# Installation
!pip install stylegan2_pytorch

In [ ]:
!stylegan2_pytorch --data ./faces --models_dir ./models --num-train-steps 35000 --num-image-tiles 1 --batch-size 6 --gradient-accumulate-every 2 --network-capacity 12
#!stylegan2_pytorch --data ./faces --name stylegan2 --models_dir ./models --num-train-steps 100 --image-size 64 --save-every 10 --evaluate-every 10

# **Inference**
**In this section, we will use trainer to train model**

In [ ]:
!stylegan2_pytorch  --generate --num_generate 1000 --image-size 64 --results_dir ./output_results --num-image-tiles 1

In [ ]:
# Rename
from PIL import Image
import os
path = './output_results/default'
files = os.listdir(path)

for index, file in enumerate(files):
    img = Image.open(os.path.join(path, file))
    img = img.resize((64,64), Image.ANTIALIAS)
    img.save(os.path.join(path, file), optimize=True)

for index, file in enumerate(files):
    if file.split("-")[-1][0] == 'm' or file.split("-")[-1][0] == 'e':
        os.remove(os.path.join(path, file))
    else:
        os.rename(os.path.join(path, file), os.path.join(path, file.split("-")[-1]))
        
os.rename(os.path.join(path, "0.jpg"), os.path.join(path, "1000.jpg"))

# for index, file in enumerate(files):
#    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index+1), '.jpg'])))

# **Prepare .tar file for submission**

In [ ]:
%cd output_results/default
!tar -zcf ../submission.tgz *.jpg